In [ ]:
#hide
from fastai.vision.all import *

# Machine Learning applications

> Showcase of typical machine learning applications leveraging transfer learning for fast implementations. 

Here we introduce some of hte most relevant applicaions of machine learning to various fields using typical academic datasets to illustrate the process. 

We leverage pre-trained models that have been trained in massive datasets for several hours in order to obtain our results. We fine-tune these models to work for our various specific tasks in order to speed up the training process by orders of magnitude and achieve state-of-the-art results. This technique is known as **transfer learning**.

## Computer vision

Computer vision is one of the most successful fields of machine learning. While in other fields, such as natural language processing, the advanaces in machine learning have taken longer, machines have long shown super-human results in the field of computer vision. For instance, a [remarkable work](https://arxiv.org/ftp/arxiv/papers/1708/1708.09843.pdf) shows how deep learning can use retinal images to detect a patient’s age, gender, smoking status and systolic blood pressure, as well as doing inference on several risk factors.

In this section, we showcase some applications of machine learning to the field of computer vision. 

### Image classification

The first task that comes into mind when we talk about computer vision is image classification. It consists on predicting a certain label for a given image among all the possibilities. 

Here, we illustrate the process with a rather funny example in which we aim to classify dog images into their respective breeds. 

In [ ]:
path = untar_data(URLs.PETS)
pets = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=RandomSplitter(seed=42),
                 get_y=using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'),
                 item_tfms=Resize(460),
                 batch_tfms=aug_transforms(size=224, min_scale=0.75))
dls = pets.dataloaders(path/"images")

In [ ]:
dls.show_batch(nrows=1, ncols=3)

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=error_rate).to_fp16()
learn.fine_tune(6, freeze_epochs=3)

#### Multi-label classification

In image classificaiton we, sometimes, encounter applications in which, rather than assigning a single label to the image, we need to provide a list of labels. This is known as multi-label classification and it is typically applied in situations in which we need to enumerate certain categories that appear in the image. I find it pretty intuitive to understand these kinds of tasks with the analogy of a kid to who we ask "what do you see in this image?" and the kid enumerates every single thing in it: a tree, a dog, the sun, a lake, grass, a house, etc. 

To provide an example, we use the [PASCAL](http://host.robots.ox.ac.uk/pascal/VOC/) dataset. 

In [ ]:
path = untar_data(URLs.PASCAL_2007)

In [ ]:
df = pd.read_csv(path/'train.csv')
df.head()

In [ ]:
def get_x(r): return path/'train'/r['fname']
def get_y(r): return r['labels'].split(' ')

def splitter(df):
    train = df.index[~df['is_valid']].tolist()
    valid = df.index[df['is_valid']].tolist()
    return train, valid

In [ ]:
dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   splitter=splitter,
                   get_x=get_x, 
                   get_y=get_y,
                   item_tfms = RandomResizedCrop(128, min_scale=0.35))
dls = dblock.dataloaders(df)

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=[accuracy_multi])

In [ ]:
learn.fine_tune(3, base_lr=3e-3, freeze_epochs=4)

### Image segmentation
In image classification we had to predict a class out of a whole image, e.g. telling whether there as a XXXXXX or a XXXXX in the image. In segmentation tasks we assign a label to each specific pixel in the image.

This technique has numerious applications in various fields. For instance, in autonomous driving we have to tell which parts of the image are road, traffic signs, pedestrians, etc. On a completely different approach, in biomedical imaging, segmentation is used to tell which regions of a given tissue are affected by certain diseases, such as telling apart tumorous cells from healthy ones, among other applications. 

Here we will show a segmentation example using the [CamVid dataset](http://www0.cs.ucl.ac.uk/staff/G.Brostow/papers/Brostow_2009-PRL.pdf) for autonomous driving. 

In [ ]:
path = untar_data(URLs.CAMVID_TINY)
dls = SegmentationDataLoaders.from_label_func(
    path, bs=8, fnames = get_image_files(path/"images"),
    label_func = lambda o: path/'labels'/f'{o.stem}_P{o.suffix}',
    codes = np.loadtxt(path/'codes.txt', dtype=str))

In [ ]:
dls.show_batch()

In [ ]:
learn = unet_learner(dls, resnet34)
learn.fine_tune(8)

In [ ]:
learn.show_results(max_n=6, figsize=(7,8))

### Image regression

The last computer vision application that we will illustrate is image regression. A regression task is characterized by assigning a real number to a sample. Hence, rather than assigning labels to the images, the model will provide a continuous value.

These kinds of tasks can be used to, for instance, infer the temperature of the soil or other atmospherical properties from satelite images or to provide the coordinates of certain objects in the given images. 

To provide an example, we will use the [Biwi kinect head pose dataset](https://icu.ee.ethz.ch/research/datsets.html), in which we want to find the coordinates of the head location of people in different videos.  

In [ ]:
path = untar_data(URLs.BIWI_HEAD_POSE)

In [ ]:
cal = np.genfromtxt(path/'01'/'rgb.cal', skip_footer=6)
def img2pose(x): return Path(f'{str(x)[:-7]}pose.txt')
def get_ctr(f):
    ctr = np.genfromtxt(img2pose(f), skip_header=3)
    c1 = ctr[0] * cal[0][0]/ctr[2] + cal[0][2]
    c2 = ctr[1] * cal[1][1]/ctr[2] + cal[1][2]
    return tensor([c1,c2])

In [ ]:
biwi = DataBlock(blocks=(ImageBlock, PointBlock),
                 get_items=get_image_files,
                 get_y=get_ctr,
                 splitter=FuncSplitter(lambda o: o.parent.name=='13'),
                 batch_tfms=[*aug_transforms(size=(240,320)), 
                             Normalize.from_stats(*imagenet_stats)])
dls = biwi.dataloaders(path)

In [ ]:
dls.show_batch(max_n=9, figsize=(8,6))

In [ ]:
xb, yb = dls.one_batch()
xb.shape, yb.shape

In [ ]:
yb[0]

In [ ]:
learn = cnn_learner(dls, resnet18, y_range=(-1,1))

In [ ]:
learn.fine_tune(3, 1e-2)

In [ ]:
learn.show_results(ds_idx=1, nrows=3, figsize=(6,8))

## Natural language processing